In [56]:
import sys, os
SRC_PATH = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
print(SRC_PATH)

/home/prashant-agrawal/Netflix_Project/src


In [57]:
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field

# ✅ Input Schema
class QdrantSearchInput(BaseModel):
    query: str = Field(..., description="User's semantic query")
    filters: Optional[Dict[str, Any]] = Field(default=None, description="Metadata filters for structured retrieval")
    k: Optional[int] = Field(default=5, description="Number of results to return")

# ✅ Output Schema
class QdrantSearchResult(BaseModel):
    results: List[str]
    message: str = ""

    def dict(self):
        return {
            "results": self.results,
            "message": self.message
        }

In [63]:
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from utils.qdrant_client_loader import get_qdrant_collection_name,get_qdrant_client


# ✅ Qdrant Search Function

def qdrant_search_fn(query: str, filters: Optional[Dict[str, Any]], k: int = 5) -> Dict:
    print(f"[Qdrant Tool] Query: {query}")
    print(f"[Qdrant Tool] Filters: {filters}")
    print(f"[Qdrant Tool] Top K: {k}")

    client = QdrantClient(url="http://localhost:6333")
    embeddings = OpenAIEmbeddings()
    COLLECTION_NAME = get_qdrant_collection_name()


    # Convert filters into Qdrant Filter format
    qdrant_filter = None
    if filters:
        conditions = []
        for key, value in filters.items():
            if isinstance(value, list):
                for v in value:
                    conditions.append(FieldCondition(key=key, match=MatchValue(value=v)))
            else:
                conditions.append(FieldCondition(key=key, match=MatchValue(value=value)))
        qdrant_filter = Filter(must=conditions)

    try:
        vectorstore = QdrantVectorStore(
            client=client,
            collection_name=COLLECTION_NAME,
            embedding=embeddings,
        )

        docs = vectorstore.similarity_search(query=query, k=k, filter=qdrant_filter)
        print(f"[Qdrant Tool] Retrieved {len(docs)} documents.")
    except Exception as e:
        print(f"[ERROR] Qdrant search failed: {e}")
        return QdrantSearchResult(results=[], message="Qdrant search failed.").dict()

    if not docs:
        return QdrantSearchResult(results=[], message="No documents found.").dict()

    return QdrantSearchResult(
        results=[doc.page_content for doc in docs],
        message=f"Top {len(docs)} documents returned."
    ).dict()




In [69]:
if __name__ == "__main__":
    query = "Top funded fintech startups in Bengaluru"
    filters = {
    "headquarters_city": "Lucknow",   # must match exactly with metadata
    "industry_sector": "Fintech"
    }

    result = qdrant_search_fn(query=query, filters=None, k=5)

    # Print results
    if result["results"]:
        for doc in result["results"]:
            print(doc)
            print("-" * 80)
    else:
        print("❌ No valid documents found.")

    print("ℹ️ Full Output:", result)

[Qdrant Tool] Query: Top funded fintech startups in Bengaluru
[Qdrant Tool] Filters: None
[Qdrant Tool] Top K: 5
[Qdrant Tool] Retrieved 5 documents.
company_name: Mobikwik
legal_entity_type: Public Ltd
state: Punjab
headquarters_city: Jaipur
year_founded: 2008
company_website: https://preston,schmidtandarcher.in
logo_url: https://logo.clearbit.com/preston,schmidtandarcher.in
company_description_short: Distributed bifurcated hub
company_description_long: Good able before computer society son above. Particularly choose owner material. Through hope partner plant tell.
Nearly computer door adult building article. Choice wide growth source if.
Month week our ready. Through response both not education data. Opportunity source near family.
industry_sector: Edtech
total_funding_raised_inr: ₹193 Cr
number_of_funding_rounds: 4
latest_funding_round_type: Series A
latest_funding_date: 2023-01-13
lead_investors: Kelley-Murphy
revenue_estimate_annual: ₹59 Cr
valuation_estimate_if_available: ₹674 Cr

In [66]:
from qdrant_client import QdrantClient
from utils.qdrant_client_loader import get_qdrant_collection_name

client = QdrantClient(url="http://localhost:6333")
collection_name = get_qdrant_collection_name()  # replace if needed

# Get a sample document
hits = client.scroll(collection_name=collection_name, limit=1)[0]
print(hits[0].payload.keys())  # what fields exist in metadata
print(hits[0].payload)  

dict_keys(['page_content', 'metadata'])
{'page_content': 'company_name: Open Financial\nlegal_entity_type: Proprietorship\nstate: Rajasthan\nheadquarters_city: Lucknow\nyear_founded: 2010\ncompany_website: https://rodriguez,burgessandwhitehead.in\nlogo_url: https://logo.clearbit.com/rodriguez,burgessandwhitehead.in\ncompany_description_short: Ameliorated static initiative\ncompany_description_long: Improve thing receive government be successful office. Federal whole trip morning boy lay evening option.\nThousand speak role type leave. Cause support father game medical trial because national. Feeling buy yet base poor team. Item join point never generation listen group.\nindustry_sector: Fintech\ntotal_funding_raised_inr: ₹226 Cr\nnumber_of_funding_rounds: 3\nlatest_funding_round_type: Seed\nlatest_funding_date: 2021-04-29\nlead_investors: Kim PLC\nrevenue_estimate_annual: ₹7 Cr\nvaluation_estimate_if_available: ₹1867 Cr\nnumber_of_employees_current: 153\nnumber_of_employees_estimate_ra